# Lesson6 ニューラルネットに画像を生成させる

In [1]:
# MNISTのロードと表示のための関数です。後で使うので読み込んでおいて下さい。
from keras.datasets import mnist
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def load_mnist(dim=3, data='mnist'):
    img_rows, img_cols = 28, 28
    
    if data == 'mnist':
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
    else:
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    
    if dim == 3:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows*img_cols)
        x_test = x_test.reshape(x_test.shape[0], img_rows*img_cols)
        
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
    y_train = np.eye(10)[y_train]
    y_test = np.eye(10)[y_test]
    
    return  x_train, x_test, y_train, y_test

def plot_mnist(n_ex=10,dim=(2,5), figsize=(8,4)):
    noise = np.random.uniform(0,1,size=[n_ex,100])
    generated_images = generator.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0],dim[1],i+1)
        img = generated_images[i,:,:, 0]
        plt.imshow(img, cmap='binary')
        plt.axis('off')
    plt.tight_layout()
    plt.show()

C:\Users\yanor\Anaconda3\envs\matsuo\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Section1 解説

### 1.1 深層生成モデル

この回では**深層生成モデル**を用いて画像を生成する。

生成モデルとは「データセットを生成する確率分布をモデル化するアプローチ」となる。今回は、深層生成モデルの中でも代表的な**Generative Adversarial Network(GAN)**とよばれるモデルを使って画像を生成する。

生成モデルを用いると、データセットが持つ抽象的な表現をモデル化しそのデータセットにありそうなデータを生成することが出来る。言葉を変えると、データセットを生成する確率分布を学習することになる。

例えば、MNISTの手書き文字のデータセットの特徴、つまり「手書きの数字」の特徴を捉え、似たような画像を生成することが出来る。

生成モデルの利用方法としては、「サンプリング」「密度推定」「欠損値補完」などがある。
- サンプリング : 確率モデルを利用して未知のデータを生成できる。
- 密度推定 : 外れ値検知や異常検知などに用いらえれる。
- 欠損値補完 : 欠損のあるデータを入力して真のデータの推定値が得られる。

等が考えられる。特にサンプリングで未知のデータを生成することから「生成モデル」とよばれている。

なお、今回は画像の生成を扱うが、生成モデル自体は画像の生成に限定されない。

まずは、ニューラルネットがMNISTの手書き文字を生成できるように学習させてみる。

### 1.2 GAN

#### 1.2.1 GANの概要

GANではGeneratorとDiscriminatorとよばれる2つのネットワークが登場する。

Generatorはデータセットと同じような画像を生成しようとする。一方、Discriminatorは入力画像がデータセットの中にある本物の画像かGeneratorが生成した偽物の画像かどうかを判定する。

GeneratorはよりDiscriminatorをだますことのできる本物に近いデータを生成しようと学習し、Discriminatorは真偽を見分けることが出来るように学習する。この両ネットワークの相乗効果によってGeneratorにはデータセットにある画像と似ている画像が生成されることが期待される。

GANは比較的くっきりとした画像が生成されることが特徴にあるが、学習が不安定なところが短所である。

![gan1](https://github.com/reminayano/matsuo/blob/master/lesson6/figures/gan1.png?raw=true)

次に具体的にどのように学習を進めていくかを数式を交えて説明する。

$$
\min_G \max_D V(D, G) = E_{x〜P_{data}(x)}[logD(x)] +  E_{z〜P_{data}(z)}[log(1-D(G(z)))]
$$

Discriminatorの出力の$D(x)$ は入力画像が訓練データのものであるかどうかの予測確率を表している。Discriminatorは$D(x)$ を最大化。また$G(z)$ は生成されたデータなので$log(1-D(G(z)))$ の最大化を行う。Generatorは逆に最小化を行う。

つまり、

1. Generatorは、学習においてDiscriminatorの学習とは独立に$V(D, G)$ を最小化するように学習する。
1. Discriminatorは、学習においてGeneratorの学習とは独立に$V(D, G)$ を最大化するように学習する。

学習を交互に行うことで両者の学習を進めていくことになる。

なお、GANの収束性については現在も研究が進められているが、依然として学習が不安定なので、ネットワークを設計する際は、論文のネットワークの形をそのまま利用したる、実際の実装コードを参考にしたりするとよい。

これから具体的な実装を見ていく。

#### 1.2.2 Generator

まず、Generatorのネットワークを構築する。

Generatorではランダムに生成したノイズからデータセットにあるような本物っぽい画像を生成することが目的。つまり、出力した画像に対してDiscriminatorが本物と勘違いするように学習する。

ここでは、要素100のノイズを入力として、サイズ28×28でチャンネル1の画像を出力している。

Generatorの中にはUpSampling2Dというレイヤーが登場するが、通常の畳み込みの逆演算として使用している。つまり、入力のwidth及びheightを増加させるように作用する。


In [2]:
from keras.layers import Input, Reshape, Dense, Flatten, Activation, Conv2D, UpSampling2D, BatchNormalization
from keras.models import Model

def Generator():
    nch = 200
    model_input = Input(shape=[100])
    x = Dense(nch*14*14, kernel_initializer='glorot_normal')(model_input) # 100 -> 200*14*14
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Reshape( [14, 14, nch] )(x) # 200*14*14 -> 14x14x200 (width)x(height)x(channel)
    x = UpSampling2D(size=(2, 2))(x) # 14x14x200 -> 28x28x200
    x = Conv2D(int(nch/2), (3, 3), padding='same', kernel_initializer='glorot_uniform')(x) # 28x28x200 -> 28x28x100
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(int(nch/4), (3, 3), padding='same', kernel_initializer='glorot_uniform')(x) # 28x28x100 -> 28x28x50
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(1, (1, 1), padding='same', kernel_initializer='glorot_uniform')(x) # 28x28x50 -> 28x28x1
    model_output = Activation('sigmoid')(x)
    model = Model(model_input, model_output)
#     model.summary()
    
    return model

#### 1.2.3 Discriminator

次にDiscriminatorのネットワークを構築する。

データセットの画像が入力されれば本物であると、Generatorによって出力されたデータであれば偽物であると判定するように学習させる。

入力としてはサイズ28×28でチャネル1の画像を入力にして、その画像がデータセットにある本物のデータかどうかを2値で出力する。

なお、DiscriminatorのActivationとしては、LeakyReLUを利用するとよい結果が出る(参照：https://arxiv.org/abs/1511.06434 )ので、LeakyReLUを使用する。(LeakyReLUはReLUの負の領域に対しても微小な勾配を可能にするReLUの特別版)

In [3]:
from keras.layers import Input, Reshape, Dense, Dropout, Flatten, LeakyReLU, Conv2D, BatchNormalization
from keras.optimizers import Adam
from keras.models import Model

def Discriminator(shape, dropout_rate=0.25, opt=Adam(lr=1e-4)):
    model_input = Input(shape=shape) # 28x28x1
    x = Conv2D(256, (5, 5), padding = 'same', kernel_initializer='glorot_uniform', strides=(2, 2))(model_input) # 28x28x1 -> 14x14x256
    x = LeakyReLU(0.2)(x)
    x = Dropout(dropout_rate)(x)
    x = Conv2D(512, (5, 5), padding = 'same', kernel_initializer='glorot_uniform', strides=(2, 2))(x) # 14x14x256 -> 7x7x512
    x = LeakyReLU(0.2)(x)
    x = Dropout(dropout_rate)(x)
    x = Flatten()(x) # 7x7x512 -> 7*7*512
    x = Dense(256)(x) # 7*7*512 -> 256
    x = LeakyReLU(0.2)(x)
    x = Dropout(dropout_rate)(x)
    model_output = Dense(2,activation='softmax')(x) # 256 -> 2
    model = Model(model_input, model_output)
    model.compile(loss='categorical_crossentropy', optimizer=opt)
    # model.summary()
    
    return model

#### 1.2.4 GANの学習

conbined_networkの関数はgeneratorの学習時に使用する。Generatorの学習ではDiscriminatorの出力を目的関数とするので、GeneratorとDiscriminatorを繋げたネットワークで学習する必要がある。

In [4]:
from keras.layers import Input
from keras.models import Model

def combined_network(generator, discriminator, opt=Adam(lr=1e-3)):
    gan_input = Input(shape=[100])
    x = generator(gan_input)
    gan_output = discriminator(x)
    model = Model(gan_input, gan_output)
    model.compile(loss='categorical_crossentropy', optimizer=opt)
    # model.summary()
    
    return model

次に、モデルの学習を制御する関数のmake_trainableを定義する。

GeneratorとDiscriminatorはそれぞれ独立に学習をするため、combined_networkでGeneratorの学習を目的とする場合、Discriminatorが一緒に学習をしないようにする必要がある。そのときに利用するのがmake_trainable関数である。

Discriminatorの各Layerのtrainableというメソッドの値をすべてFalseにすることができ、パラメータを更新しないようにすることができる。この後のtrain関数内で使用する。

In [5]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

以下のtrain関数で実際に学習を進めていく。

train関数内では以下のような学習を各バッチごとに行う。

1. バッチの学習で利用する画像の選択
1. Discriminatorの学習をonに切り替える
1. Generatorによる学習をonに切り替える
1. Discriminatorの学習をoffに切り替える
1. Generatorの学習


In [6]:
from tqdm import tqdm

def train(step=3000, BATCH_SIZE=128):
    for e in tqdm(range(step)):
        # 1. バッチの学習で利用する画像の選択 
        # バッチサイズの分だけランダムに画像を選択
        image_batch = X_train[np.random.randint(0,X_train.shape[0],size=BATCH_SIZE),:,:,:]
        
        # バッチサイズの分だけランダムにノイズを生成し、generatorにより画像を生成
        noise_gen = np.random.uniform(0,1,size=[BATCH_SIZE,100])
        generated_images = generator.predict(noise_gen)
        
        # 2. Discriminatorの学習をonに切り替える
        # Discriminatorが学習するように変更
        make_trainable(discriminator,True)
        
        # 3. Generatorによる生成画像を用いてDiscriminatorの学習
        # X = (バッチサイズ分のデータセットの画像, バッチサイズ分の生成画像)
        X = np.concatenate((image_batch, generated_images))
        
        # y = (バッチサイズ分のTrue(本物), バッチサイズ分のFalse(偽物))
        y = np.zeros([2*BATCH_SIZE,2])
        y[:BATCH_SIZE,1] = 1
        y[BATCH_SIZE:,0] = 1      
        
        # Discriminatorのtrain
        discriminator.train_on_batch(X,y)
        
        # 4. Discriminatorの学習をoffに切り替える
        # Discriminatorが学習しないように変更
        make_trainable(discriminator,False)
    
        # 5. Generatorの学習
        # バッチサイズの分だけランダムにノイズを生成
        noise_gen = np.random.uniform(0,1,size=[BATCH_SIZE,100])
        
        # y = (バッチサイズ分のTrue(本物))
        # 実際には生成した画像なのでDiscriminatorとしては偽物と判断すべきだが、Genaratorの学習なので生成した画像を本物と判断するように学習させる
        y2 = np.zeros([BATCH_SIZE,2])
        y2[:,1] = 1
        
        # Generatorのtrain
        GAN.train_on_batch(noise_gen, y2 )


#### 1.2.5 MNISTによる学習

In [7]:
# データのロード
X_train, _,_,_ = load_mnist()
# それぞれのネットワークのインスタンスを生成
generator = Generator()
discriminator = Discriminator(X_train.shape[1:])
make_trainable(discriminator, False)
GAN = combined_network(generator, discriminator)

In [ ]:
# このセルは実行時間長いのでやってない(24時間以上？)
# train関数で学習を行うstepを大きくすると学習をより多く行います

train()

#### 1.2.6 手書き文字の生成

学習が完了したら、実際にランダムのノイズから手書き文字のような画像が出力されるかを表示して確認してみる。

In [ ]:
plot_mnist()